<a href="https://colab.research.google.com/github/Aditikumari20/NLP/blob/main/12_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.layers import Input, Dense, Dropout, LeakyReLU
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [7]:
np.random.seed(10)
random_dim = 100

In [8]:
def load_mnist_data():
  (x_train, y_train), (x_test, y_test) = mnist.loaad_data(_)
  x_train = (x_train.astype(np.float32) - 127.5)/127.5
  x_train = x_train.reshape(60000, 784)
  return (x_train, y_train, x_test, y_test)


In [9]:
def get_optimizer():
  return Adam(learning_rate=0.0002, beta_1=0.5)

In [10]:
def get_generator(optimizer):
  generator = Sequential()
  generator.add(Dense(256, input_dim=random_dim, \
                      kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(784, activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return generator



In [11]:
def get_discriminator(optimizer):
  discriminator = Sequential()
  discriminator.add(Dense(256, input_dim=random_dim, \
                      kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(1024))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(784, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return discriminator


In [12]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable = False
  gan_input = Input(shape=(random_dim,))
  x = generator(gan_input)
  gan_ouput = discriminator(x)
  gan = Model(inputs=gan_input, outputs=gan_ouput)
  gan.compile(loss='binary_crossentropy', optimizer=optimizer)
  return gan

In [1]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), \
                          figsize=(10, 10)):
  noise = np.random.normal(0, 1, size=[examples, random_dim])
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(examples, 28, 28)

  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation='nearest', \
               cmap='gray_r')
    plt.axis('off')
  plt.tight_layout()
  plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [5]:
def train(epochs=1, batch_size=128):
    # 1. Get the training and testing data
    x_train, y_train, x_test, y_test = load_mnist_data()  # typo here, should be load_mnist_data
    # Split the training data into batches of size 128
    batch_count = x_train.shape[0] / batch_size
    # 2. Build our GAN network
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)  # typo here, should be gan, and generator
    # 3. Training loop for a specified number of epochs
    for e in range(1, epochs + 1):
        print('-' * 15, 'Epoch %d' % e, '-' * 15)
        for _ in tqdm(range(int(batch_count))):
            # 4. Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
            # 5. Generate fake MNIST images
            generated_images = generator.predict(noise)
            # 6. Combine real and fake images for discriminator training
            X = np.concatenate([image_batch, generated_images])  # missing bracket here
            # 7. Create labels for discriminator (0 for fake, 1 for real)
            y_dis = np.zeros(2 * batch_size)
            # One-sided label smoothing, real images labeled as 0.9 instead of 1
            y_dis[:batch_size] = 0.9
            # 8. Train the discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)
            # 9. Train the generator (through the GAN network)
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)  # Labels for generator (all 1s, trying to fool discriminator)
            discriminator.trainable = False  # Freeze discriminator weights during generator training
            gan.train_on_batch(noise, y_gen)
        # 10. Plot generated images at specific epochs
        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)

In [ ]:
train(20,128)